In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## class definitions

In [21]:
class RansacPodDetector:
    def __init__(self):
        pass

    def _preprocess(self, pcd : o3d.geometry.PointCloud, voxel_size: float):
        pcd_down_sampled = pcd.voxel_down_sample(voxel_size)
        pcd_down_sampled.normal_estimate()
        
        return pcd_down_sampled

    def _extract_fpfh_features(self, pcd : o3d.geometry.PointCloud, voxel_size: float):
        fpfh_search_radius = 10 * voxel_size # default is 5
        fpfh_max_nearest_neighbor = 30
        
        fpfh_features =  o3d.pipelines.registration.compute_fpfh_feature(
            pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=fpfh_search_radius, max_nn=fpfh_max_nearest_neighbor)
        )

        return fpfh_features

    def _ransac_registration(self, pcd_source: o3d.geometry.PointCloud, pcd_target: o3d.geometry.PointCloud, 
                             fpfh_source, fpfh_target, voxel_size: float):
        distance_threshold = 1.5 * voxel_size # default is 1.5
        estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint(False) # disable scalling
        use_ratio_test = True
        enable_checkers = [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(np.pi / 2) # ignore if the angle between two normals are over 90 deg 
        ]
        criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)

        ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            pcd_source, pcd_target, fpfh_source, fpfh_target, use_ratio_test, distance_threshold,
            estimation_method, ransac_n=3, checkers=enable_checkers, criteria=criteria
        )

        return ransac_result
        
    def detect(self, pcd_source, pcd_target, voxel_size=0.005, visualize=False):
        # down sampling
        pcd_source_down_sampled = pcd_source.voxel_down_sample(voxel_size)
        pcd_target_down_sampled = pcd_target.voxel_down_sample(voxel_size)
        
        ## normals estimation
        # this makes normals look inside (TODO : make nomals look outside)
        #pcd_source_down_sampled.estimate_normals()
        #pcd_source_down_sampled.orient_normals_consistent_tangent_plane(k=15)
        pcd_target_down_sampled.estimate_normals()
        pcd_target_down_sampled.orient_normals_consistent_tangent_plane(k=15)

        # fpfh feature extraction
        fpfh_source = self._extract_fpfh_features(pcd_source_down_sampled, voxel_size)
        fpfh_target = self._extract_fpfh_features(pcd_target_down_sampled, voxel_size)

        # ransac
        result = self._ransac_registration(pcd_source_down_sampled, pcd_target_down_sampled,
                                         fpfh_source, fpfh_target, voxel_size)

        # display the ransac result
        if visualize == True:
            pcd_source_down_sampled.paint_uniform_color([1, 0, 0])
            pcd_target_down_sampled.paint_uniform_color([0, 1, 0])
            self.draw_registration_result([pcd_source_down_sampled], [pcd_target_down_sampled], result.transformation)
        
        return result

    def draw_registration_result(self, source, target, transformation):
        pcds = list()
        for s in source:
            temp = copy.deepcopy(s)
            pcds.append(temp.transform(transformation))
        pcds += target
        o3d.visualization.draw_geometries(pcds)

In [3]:
class HelperCollection:
    def __init__(self):
        import copy

    def load_realsense_pcd(file_number, visualize = False):
        file_path = '../dataset/d405/d405-' + str(file_number) + '.ply'
        print(f'||| loading {file_path} ...')
        pcd_source = o3d.io.read_point_cloud(file_path)
        print(f'||| loading complete')
        
        if visualize == True:
            mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
            o3d.visualization.draw_geometries([pcd_source, mesh_frame])

        return pcd_source
    
    def load_cad_model_pcd(visualize = False):
        file_path = '../dataset/model/art_melon_without_handle.STL'
        print(f'||| loading {file_path}')
        mesh_target = o3d.io.read_triangle_mesh(file_path)
        print(f'||| loading complete')
        mesh_target.compute_triangle_normals()
        pcd_target = mesh_target.sample_points_poisson_disk(number_of_points=30000)
        
        if visualize == True:
            mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
            o3d.visualization.draw_geometries([mesh_target, mesh_frame])

        return pcd_target

    def crop_arround_pot(pcd_source):
        tmp_pcd_source = copy.deepcopy(pcd_source)
        
        # crop only bottle
        min_bound = np.asarray([-0.5, -1.0, -1.0]) # [x_min, y_min, z_min]
        max_bound = np.asarray([0.25, 1.0, 1.0]) # [x_max, y_max, z_max]
        aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        pcd_source_cropped = tmp_pcd_source.crop(aabb)
        print(f'||| croping arround pot by aabb')
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
        o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

        # remove table
        plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=500)
        pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
        pcd_plane.paint_uniform_color([1, 0, 0])
        o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
        pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
        print('||| estimate plane and remove it')
        o3d.visualization.draw_geometries([pcd_source_removed_plane, mesh_frame])

        # remove outlier
        cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=60, std_ratio=2.5)
        pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
        print('||| remove outlier')
        o3d.visualization.draw_geometries([pcd_source_removed_outlier, mesh_frame])

        return pcd_source_removed_outlier

    def rotate_y_axis_to_upper(pcd_cad_model: o3d.geometry.PointCloud):
        tmp_pcd_cad_model = copy.deepcopy(pcd_cad_model)
        
        rad = np.radians(-90)
        R = o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([rad, 0, 0]))
        tmp_pcd_cad_model.rotate(R)
        
        return tmp_pcd_cad_model

    def move_centroid_to_origin(pcd):
        tmp_pcd = copy.deepcopy(pcd) 
        tmp_pcd.translate(-tmp_pcd.get_center())
        return tmp_pcd

## test ransac class

#### 1. load data

In [4]:
# load cad model as pcd
pcd_target = HelperCollection.load_cad_model_pcd(visualize=True)

||| loading ../dataset/model/art_melon_without_handle.STL
||| loading complete


In [5]:
# load realsense ovservation as pcd
pcd_source = HelperCollection.load_realsense_pcd(file_number=2, visualize=True)

||| loading ../dataset/d405/d405-2.ply ...
||| loading complete


#### 2. preprocess (rotate and so on)

In [6]:
# crop arround pot
pcd_source_cropped = HelperCollection.crop_arround_pot(pcd_source)

||| croping arround pot by aabb
||| estimate plane and remove it
||| remove outlier


In [7]:
# rotate the cad model -90deg
pcd_target_rotated = HelperCollection.rotate_y_axis_to_upper(pcd_target)
pcd_target_aligned = HelperCollection.move_centroid_to_origin(pcd_target_rotated)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
o3d.visualization.draw_geometries([pcd_target_aligned, mesh_frame])

In [8]:
# move the cad model centroid to observation centroid
pcd_target_aligned.translate(pcd_source_cropped.get_center())
o3d.visualization.draw_geometries([pcd_target_aligned, pcd_source_cropped, mesh_frame])

#### 3. ransac

In [22]:
detector = RansacPodDetector()

In [31]:
# 0.01 for art-melon pot
voxel_size = 0.010
result = detector.detect(pcd_source_cropped, pcd_target_aligned, voxel_size, visualize=True)
#detector.draw_registration_result([pcd_source_cropped], [pcd_target_aligned], result.transformation)

[Open3D WARNING] Too few correspondences (36) after mutual filter, fall back to original correspondences.


In [19]:
result.transformation

array([[ 0.67453075, -0.20770981,  0.70842424,  0.2723103 ],
       [ 0.17666055,  0.97713815,  0.11828816,  0.03352446],
       [-0.71679796,  0.04536161,  0.69580386, -0.05158998],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])